In [1]:
using DifferentialEquations
using Printf
using CairoMakie
using LaTeXStrings
using NumericalIntegration
using DelimitedFiles
using JLD2

In [2]:
#Constant information, can be global, but storing in a dictionary to keep tidy

constants = Dict(
"T"   => 298,            #K         
"P"   => 1.0,            #bar
"P0"  => 1.0,            #bar
"MC"  => 10.0,           #grams of catalyst in reactor
"FAf" => 1,              #mol/s
"FBf" => 2,              #mol/s
"FCf" => 0,              #mol/s
"FIf" => 1000,           #mol/s
"FPf" => 0,              #mol/s
"R"   => 8.314,          #J/mol/K
"kb"  => 1.380649e-23,   #J/K
"h"   => 6.62607015e-34, #J*s
"φ"   => 0.5,            #V_void/V_total
"VR"  => 50000.0,        #L
"VF"  => 50000.0*0.5,     #L
"VC"  => 50000.0*0.5,    #L
)

Dict{String, Real} with 16 entries:
  "VC"  => 25000.0
  "FPf" => 0
  "T"   => 298
  "P"   => 1.0
  "MC"  => 10.0
  "FBf" => 2
  "h"   => 6.62607e-34
  "P0"  => 1.0
  "FAf" => 1
  "kb"  => 1.38065e-23
  "φ"   => 0.5
  "FCf" => 0
  "VF"  => 25000.0
  "FIf" => 1000
  "VR"  => 50000.0
  "R"   => 8.314

In [3]:
function tCSTR1!(du, u, p, t)
   
    #unpack parameters
    FAf_S, FBf_S, FCf_S, FPf_S, FIf_S, T, P, P0, kf, kr, τF = p
    
    #Labelling unknowns for readable equations; Fj [=] mol/s/mol site
    FA_S = u[1]
    FB_S = u[2]
    FC_S = u[3]
    FP_S = u[4]
    θA   = u[5]
    θB   = u[6]
    θC   = u[7]
    θP   = u[8]
    
    #Calculate vacancies from site balance
    θV   = 1 - θA - θB - θC - θP
    
    #Total molar flowrate of gas phase species
    FT_S   = FA_S + FB_S + FC_S + FP_S + FIf_S #mol/s/mol site
    
    #Mole fractions
    yA     = FA_S/FT_S
    yB     = FB_S/FT_S
    yC     = FC_S/FT_S
    yP     = FP_S/FT_S
    
    #Thermodynamic activities
    aA     = yA*P/P0
    aB     = yB*P/P0
    aC     = yC*P/P0
    aP     = yP*P/P0
    
    #Calculate rate constants at system temperature
    k1f, k2f, k3f, k4f, k5f = kf
    k1r, k2r, k3r, k4r, k5r = kr
    
    #Elementary reaction rates in TOF units
    r1 = k1f*aA*θV - k1r*θA
    r2 = k2f*aB*θV - k2r*θB
    r3 = k3f*aC*θV - k3r*θC
    r4 = k4f*θA*θB - k4r*θC*θV
    r5 = 0

    #Production rates, TOF units [=] moles species/mole site/time
    RA     = -r1
    RB     = -r2
    RC     = -r3
    RP     =  0
    RAs    =  r1 - r4
    RBs    =  r2 - r4
    RCs    =  r3 + r4
    RPs    =  0
    
    #Material balances, LHS = ODE 
    du[1]     = 1/τF*(FAf_S - FA_S + RA) #[=] moles species/mole site/time/time
    du[2]     = 1/τF*(FBf_S - FB_S + RB) #[=] moles species/mole site/time/time
    du[3]     = 1/τF*(FCf_S - FC_S + RC) #[=] moles species/mole site/time/time
    du[4]     = 1/τF*(FPf_S - FP_S + RP) #[=] moles species/mole site/time/time
    du[5]     = RAs #[=] moles species/mole site/time
    du[6]     = RBs #[=] moles species/mole site/time
    du[7]     = RCs #[=] moles species/mole site/time
    du[8]     = RPs #[=] moles species/mole site/time
end

tCSTR1! (generic function with 1 method)

In [4]:
function tCSTRmain1(constants, sim_pars)
    
    #Pull constants out of dictionary of parameters
    T   = constants["T"]
    P   = constants["P"]
    P0  = constants["P0"]
    FAf = constants["FAf"]
    FBf = constants["FBf"]
    FCf = constants["FCf"]
    FIf = constants["FIf"]
    FPf = constants["FPf"]
    R   = constants["R"]
    kb  = constants["kb"]
    h   = constants["h"]
    φ   = constants["φ"]
    VR  = constants["VR"]
    VF  = constants["VF"]
    VC  = constants["VC"]
    
    #Experiment/Simulation Specific Information
    NS0    = sim_pars["NS0"]    #moles of active sites in bed  
    DSACT  = sim_pars["DSACT"]  #Entropy of activation, J/mol/K
    DHACT  = sim_pars["DHACT"]  #Enthalpy of activation, J/mol
    DS     = sim_pars["DS"]     #Entropy of reaction, J/mol/K
    DH     = sim_pars["DH"]     #Enthalpy of reaction, J/mol
    
    #Calculating quantities relevant to this simulation
    FAf_S  = FAf/NS0        #mol/s/mol active site
    FBf_S  = FBf/NS0        #mol/s/mol active site
    FCf_S  = FCf/NS0        #mol/s/mol active site
    FPf_S  = FPf/NS0
    FIf_S  = FIf/NS0        #mol/s/mol active site

    #Temperature dependent quantities
    K     = exp.(DS/R).*exp.(-DH/R/T)
    kf    = kb*T/h*exp.(DSACT/R).*exp.(-DHACT/R/T)
    kr    = kf./K
    Qf    = (FAf + FBf + FCf + FIf)*0.083145*T/P #L/s
    τF    = VF/Qf
    
    #Set up ODE solver, solve problem
    par = FAf_S, FBf_S, FCf_S, FPf_S, FIf_S, T, P, P0, kf, kr, τF
    domainfunc = (u, p, t) -> any(x -> x < 0, u)
    u0 = [0., 0., 0., 0., 0., 0., 0., 0.] 
    tspan = (0.0, 1e13) #adequate time to reach steady state? t --> :∞ 
    tvals   = 10 .^range(-1, 13, length = 200)
    prob = ODEProblem(tCSTR1!, u0, tspan, par)  
    tCSTRsol = solve(prob, Rodas4(), isoutofdomain = domainfunc)
    
    #Benchmarking results
    #display(@benchmark solve($prob, Rosenbrock23(), isoutofdomain = $domainfunc, saveat = $tvals))
    
    #time set for plotting on log x axis
    tCSTRout = tCSTRsol(tvals)
    
    #Output information, function of time.
    FA_S = tCSTRout[1, :]
    FB_S = tCSTRout[2, :]
    FC_S = tCSTRout[3, :]
    FP_S = tCSTRout[4, :]
    θA   = tCSTRout[5, :]
    θB   = tCSTRout[6, :]
    θC   = tCSTRout[7, :]
    θP   = tCSTRout[8, :]
    θV   = 1 .- θA .- θB .- θC .- θP
    XA   = (FAf_S .- FA_S)/FAf_S
    
    #Specify returns
    return tvals, FA_S, FB_S, FC_S, FP_S, XA, θA, θB, θC, θP, θV
end

tCSTRmain1 (generic function with 1 method)

In [5]:
#Solve the problem, store the returns.
sim_pars1 = Dict(
    "NS0"   => 10,                           #moles of active sites in bed
    "DSACT" => [-100, -100, -100, 0, 0],      #J/mol/K
    "DHACT" => [0, 0, 0, 100, 130]*1000,      #J/mol
    "DS"    => [-100, -100, -100, 0, 0],      #J/mol/K
    "DH"    => [-100, -110, -100, 0, -5]*1000, #J/mol
)

t1, FA_S1, FB_S1, FC_S1, FP_S1, XA1, θA1, θB1, θC1, θP1, θV1 = tCSTRmain1(constants, sim_pars1);
tLABEL1 = ["t (s)" "FA (mol/s/mol site)" "FB (mol/s/mol site)" "FC (mol/s/mol site)" "FP (mol/s/mol site)" "XA" "θA" "θB" "θC" "θP" "θV"]
tDATA1  = [t1/1e6 FA_S1*1e6 FB_S1*1e6 FC_S1*1e6 FP_S1 XA1 θA1 θB1 θC1 θP1 θV1]
FLOW_TRANSIENT_1 = vcat(tLABEL1, tDATA1);

io = open("FLOW_TRANSIENT_1.csv", "w")
writedlm(io, FLOW_TRANSIENT_1, ',')
close(io)

@printf("Steady state flowrates of A, B, and C are %3.2E, %3.2E, %3.2E mol/s/mol active site \n", FA_S1[end], FB_S1[end], FC_S1[end])
@printf("Fractional Conversion of A is %3.2E \n", XA1[end])
@printf("Steady State Coverages of A, B, C, P, and V are %3.2E, %3.2E, %3.2E, %3.2E, %3.2E \n", θA1[end], θB1[end], θC1[end], θP1[end], θV1[end])
@printf("The turnover frequency for this reaction is approximately %3.2E mol/s/mol site.", FC_S1[end])

Steady state flowrates of A, B, and C are 1.00E-01, 2.00E-01, 1.06E-07 mol/s/mol active site 
Fractional Conversion of A is 1.06E-06 
Steady State Coverages of A, B, C, P, and V are 5.84E-03, 9.88E-01, 5.77E-03, 0.00E+00, 5.77E-12 
The turnover frequency for this reaction is approximately 1.06E-07 mol/s/mol site.

In [6]:
function tCSTR2!(du, u, p, t)
   
    #unpack parameters
    FAf_S, FBf_S, FCf_S, FPf_S, FIf_S, T, P, P0, kf, kr, τF = p
    
    #Labelling unknowns for readable equations; Fj [=] mol/s/mol site
    FA_S = u[1]
    FB_S = u[2]
    FC_S = u[3]
    FP_S = u[4]
    θA   = u[5]
    θB   = u[6]
    θC   = u[7]
    θP   = u[8]
    
    #Calculate vacancies from site balance
    θV   = 1 - θA - θB - θC - θP
    
    #Total molar flowrate of gas phase species
    FT_S   = FA_S + FB_S + FC_S + FP_S + FIf_S #mol/s/mol site
    
    #Mole fractions
    yA     = FA_S/FT_S
    yB     = FB_S/FT_S
    yC     = FC_S/FT_S
    yP     = FP_S/FT_S
    
    #Thermodynamic activities
    aA     = yA*P/P0
    aB     = yB*P/P0
    aC     = yC*P/P0
    aP     = yP*P/P0
    
    #Calculate rate constants at system temperature
    k1f, k2f, k3f, k4f, k5f = kf
    k1r, k2r, k3r, k4r, k5r = kr
    
    #Elementary reaction rates in TOF units
    r1 = k1f*aA*θV - k1r*θA
    r2 = k2f*aB*θV - k2r*θB
    r3 = k3f*aC*θV - k3r*θC
    r4 = k4f*θA*θB - k4r*θC*θV
    r5 = k5f*θA    - k5r*θP

    #Production rates, TOF units [=] moles species/mole site/time
    RA     = -r1
    RB     = -r2
    RC     = -r3
    RP     =  0
    RAs    =  r1 - r4 - r5
    RBs    =  r2 - r4
    RCs    =  r3 + r4
    RPs    =  r5
    
    #Material balances, LHS = ODE 
    du[1]     = 1/τF*(FAf_S - FA_S + RA) #[=] moles species/mole site/time/time
    du[2]     = 1/τF*(FBf_S - FB_S + RB) #[=] moles species/mole site/time/time
    du[3]     = 1/τF*(FCf_S - FC_S + RC) #[=] moles species/mole site/time/time
    du[4]     = 1/τF*(FPf_S - FP_S + RP) #[=] moles species/mole site/time/time
    du[5]     = RAs #[=] moles species/mole site/time
    du[6]     = RBs #[=] moles species/mole site/time
    du[7]     = RCs #[=] moles species/mole site/time
    du[8]     = RPs #[=] moles species/mole site/time
end

tCSTR2! (generic function with 1 method)

In [7]:
function tCSTRmain2(constants, sim_pars)
    
    #Pull constants out of dictionary of parameters
    T   = constants["T"]
    P   = constants["P"]
    P0  = constants["P0"]
    FAf = constants["FAf"]
    FBf = constants["FBf"]
    FCf = constants["FCf"]
    FIf = constants["FIf"]
    FPf = constants["FPf"]
    R   = constants["R"]
    kb  = constants["kb"]
    h   = constants["h"]
    φ   = constants["φ"]
    VR  = constants["VR"]
    VF  = constants["VF"]
    VC  = constants["VC"]
    
    #Experiment/Simulation Specific Information
    NS0    = sim_pars["NS0"]    #moles of active sites in bed  
    DSACT  = sim_pars["DSACT"]  #Entropy of activation, J/mol/K
    DHACT  = sim_pars["DHACT"]  #Enthalpy of activation, J/mol
    DS     = sim_pars["DS"]     #Entropy of reaction, J/mol/K
    DH     = sim_pars["DH"]     #Enthalpy of reaction, J/mol
    
    #Calculating quantities relevant to this simulation
    FAf_S  = FAf/NS0        #mol/s/mol active site
    FBf_S  = FBf/NS0        #mol/s/mol active site
    FCf_S  = FCf/NS0        #mol/s/mol active site
    FPf_S  = FPf/NS0
    FIf_S  = FIf/NS0        #mol/s/mol active site

    #Temperature dependent quantities
    K     = exp.(DS/R).*exp.(-DH/R/T)
    kf    = kb*T/h*exp.(DSACT/R).*exp.(-DHACT/R/T)
    kr    = kf./K
    Qf    = (FAf + FBf + FCf + FIf)*0.083145*T/P #L/s
    τF    = VF/Qf
    
    #Set up ODE solver, solve problem
    par = FAf_S, FBf_S, FCf_S, FPf_S, FIf_S, T, P, P0, kf, kr, τF
    domainfunc = (u, p, t) -> any(x -> x < 0, u)
    u0 = [0., 0., 0., 0., 0., 0., 0., 0.] 
    tspan = (0.0, 1e13) #adequate time to reach steady state? t --> :∞ 
    tvals   = 10 .^range(-1, 13, length = 200)
    prob = ODEProblem(tCSTR2!, u0, tspan, par)  
    tCSTRsol = solve(prob, Rodas4(), isoutofdomain = domainfunc)
    
    #Benchmarking results
    #display(@benchmark solve($prob, Rosenbrock23(), isoutofdomain = $domainfunc, saveat = $tvals))
    
    #time set for plotting on log x axis
    tCSTRout = tCSTRsol(tvals)
    
    #Output information, function of time.
    FA_S = tCSTRout[1, :]
    FB_S = tCSTRout[2, :]
    FC_S = tCSTRout[3, :]
    FP_S = tCSTRout[4, :]
    θA   = tCSTRout[5, :]
    θB   = tCSTRout[6, :]
    θC   = tCSTRout[7, :]
    θP   = tCSTRout[8, :]
    θV   = 1 .- θA .- θB .- θC .- θP
    XA   = (FAf_S .- FA_S)/FAf_S
    
    #Specify returns
    return tvals, FA_S, FB_S, FC_S, FP_S, XA, θA, θB, θC, θP, θV
end

tCSTRmain2 (generic function with 1 method)

In [8]:
sim_pars2 = Dict(
    "NS0"   => 10,                           #moles of active sites in bed
    "DSACT" => [-100, -100, -100, 0, 0],      #J/mol/K
    "DHACT" => [0, 0, 0, 100, 105]*1000,      #J/mol
    "DS"    => [-100, -100, -100, 0, 0],      #J/mol/K
    "DH"    => [-100, -100, -100, 0, -5]*1000, #J/mol
)

t2, FA_S2, FB_S2, FC_S2, FP_S2, XA2, θA2, θB2, θC2, θP2, θV2 = tCSTRmain2(constants, sim_pars2);
tLABEL2 = ["t (s)" "FA (mol/s/mol site)" "FB (mol/s/mol site)" "FC (mol/s/mol site)" "FP (mol/s/mol site)" "XA" "θA" "θB" "θC" "θP" "θV"]
tDATA2  = [t2/1e6 FA_S2*1e6 FB_S2*1e6 FC_S2*1e6 FP_S2 XA2 θA2 θB2 θC2 θP2 θV2]
FLOW_TRANSIENT_2 = vcat(tLABEL2, tDATA2);

io = open("FLOW_TRANSIENT_2.csv", "w")
writedlm(io, FLOW_TRANSIENT_2, ',')
close(io)

@printf("Steady state flowrates of A, B, and C are %3.2E, %3.2E, %3.2E mol/s/mol active site \n", FA_S2[end], FB_S2[end], FC_S2[end])
@printf("Fractional Conversion of A is %3.2E \n", XA2[end])
@printf("Steady State Coverages of A, B, C, P, and V are %3.2E, %3.2E, %3.2E, %3.2E, %3.2E \n", θA2[end], θB2[end], θC2[end], θP2[end], θV2[end])
@printf("The turnover frequency for this reaction is approximately %3.2E mol/s/mol site.", FC_S2[end])

Steady state flowrates of A, B, and C are 1.00E-01, 2.00E-01, 3.39E-07 mol/s/mol active site 
Fractional Conversion of A is 3.39E-06 
Steady State Coverages of A, B, C, P, and V are 9.15E-02, 2.01E-01, 1.84E-02, 6.89E-01, 5.46E-11 
The turnover frequency for this reaction is approximately 3.39E-07 mol/s/mol site.

In [9]:
t3, FA_S3, FB_S3, FC_S3, FP_S3, XA3, θA3, θB3, θC3, θP3, θV3 = tCSTRmain1(constants, sim_pars2);
tLABEL3 = ["t (s)" "FA (mol/s/mol site)" "FB (mol/s/mol site)" "FC (mol/s/mol site)" "FP (mol/s/mol site)" "XA" "θA" "θB" "θC" "θP" "θV"]
tDATA3  = [t3/1e6 FA_S3*1e6 FB_S3*1e6 FC_S3*1e6 FP_S3 XA3 θA3 θB3 θC3 θP3 θV3]
FLOW_TRANSIENT_3 = vcat(tLABEL3, tDATA3);

io = open("FLOW_TRANSIENT_3.csv", "w")
writedlm(io, FLOW_TRANSIENT_3, ',')
close(io)

@printf("Steady state flowrates of A, B, and C are %3.3E, %3.3E, %3.3E mol/s/mol active site \n", FA_S3[end], FB_S3[end], FC_S3[end])
@printf("Fractional Conversion of A is %3.3E \n", XA3[end])
@printf("Steady State Coverages of A, B, C, P, and V are %3.3E, %3.3E, %3.3E, %3.3E, %3.3E \n", θA3[end], θB3[end], θC3[end], θP3[end], θV3[end])
@printf("The turnover frequency for this reaction is approximately %3.3E mol/s/mol site.", FC_S3[end])

Steady state flowrates of A, B, and C are 1.000E-01, 2.000E-01, 2.679E-06 mol/s/mol active site 
Fractional Conversion of A is 2.679E-05 
Steady State Coverages of A, B, C, P, and V are 2.361E-01, 6.180E-01, 1.459E-01, 0.000E+00, 1.896E-10 
The turnover frequency for this reaction is approximately 2.679E-06 mol/s/mol site.

In [10]:
function tCSTR4!(du, u, p, t)

    #unpack parameters
    FAf = p["FAf"]
    FBf = p["FBf"]
    FCf = p["FCf"]
    FPf = p["FPf"]
    FIf = p["FIf"]
    T   = p["T"]
    P   = p["P"]
    P0  = p["P0"]
    kf  = p["kf"]
    kr  = p["kr"]
    NS0 = p["NS0"]
    τF  = p["τF"]
    
    #Labelling unknowns for readable equations; Fj [=] mol/s/mol site
    FA = u[1]
    FB = u[2]
    FC = u[3]
    FP = u[4]
    θA = u[5]
    θB = u[6]
    θC = u[7]
    θP = u[8]
    NS = NS0 + 365*(1 - exp(-10000*t/1e10))
    
    #Calculate vacancies from site balance
    θV   = 1 - θA - θB - θC - θP
    
    #Total molar flowrate of gas phase species
    FT   = FA + FB + FC + FP + FIf #mol/s/mol site
    
    #Mole fractions
    yA     = FA/FT
    yB     = FB/FT
    yC     = FC/FT
    yP     = FP/FT
    
    #Thermodynamic activities
    aA     = yA*P/P0
    aB     = yB*P/P0
    aC     = yC*P/P0
    aP     = yP*P/P0
    
    #Calculate rate constants at system temperature
    k1f, k2f, k3f, k4f, k5f = kf
    k1r, k2r, k3r, k4r, k5r = kr
    
    #Elementary reaction rates in TOF units
    r1 = k1f*aA*θV - k1r*θA
    r2 = k2f*aB*θV - k2r*θB
    r3 = k3f*aC*θV - k3r*θC
    r4 = k4f*θA*θB - k4r*θC*θV
    r5 = 1e15*aP*θV

    #Production rates, TOF units [=] moles species/mole site/time
    RA     = -r1
    RB     = -r2
    RC     = -r3
    RP     = -r5
    RAs    =  r1 - r4
    RBs    =  r2 - r4
    RCs    =  r3 + r4
    RPs    =  r5
    
    #Material balances, LHS = ODE 
    du[1]     = 1/τF*(FAf - FA + RA*NS) #[=] moles species/mole site/time/time
    du[2]     = 1/τF*(FBf - FB + RB*NS) #[=] moles species/mole site/time/time
    du[3]     = 1/τF*(FCf - FC + RC*NS) #[=] moles species/mole site/time/time
    du[4]     = 1/τF*(FPf - FP + RP*NS) #[=] moles species/mole site/time/time
    du[5]     = RAs #[=] moles species/mole site/time
    du[6]     = RBs #[=] moles species/mole site/time
    du[7]     = RCs #[=] moles species/mole site/time
    du[8]     = RPs #[=] moles species/mole site/time
end

tCSTR4! (generic function with 1 method)

In [11]:
function tCSTRmain4(constants, sim_pars)
    
    #Pull constants out of dictionary of parameters
    T   = constants["T"]
    P   = constants["P"]
    P0  = constants["P0"]
    FAf = constants["FAf"]
    FBf = constants["FBf"]
    FCf = constants["FCf"]
    FIf = constants["FIf"]
    FPf = constants["FPf"]
    R   = constants["R"]
    kb  = constants["kb"]
    h   = constants["h"]
    φ   = constants["φ"]
    VR  = constants["VR"]
    VF  = constants["VF"]
    VC  = constants["VC"]
    
    #Experiment/Simulation Specific Information
    NS0    = sim_pars["NS0"]    #moles of active sites in bed  
    DSACT  = sim_pars["DSACT"]  #Entropy of activation, J/mol/K
    DHACT  = sim_pars["DHACT"]  #Enthalpy of activation, J/mol
    DS     = sim_pars["DS"]     #Entropy of reaction, J/mol/K
    DH     = sim_pars["DH"]     #Enthalpy of reaction, J/mol
    cb     = sim_pars["CB"]     #callback function
        
    #Temperature dependent quantities
    K     = exp.(DS/R).*exp.(-DH/R/T)
    kf    = kb*T/h*exp.(DSACT/R).*exp.(-DHACT/R/T)
    kr    = kf./K
    Qf    = (FAf + FBf + FCf + FIf)*0.083145*T/P #L/s
    τF    = VF/Qf
    
    #Set up ODE solver, solve problem
    par = Dict(
        "FAf" => FAf, 
        "FBf" => FBf, 
        "FCf" => FCf, 
        "FPf" => FPf, 
        "FIf" => FIf, 
        "T"   => T, 
        "P"   => P, 
        "P0"  => P0, 
        "kf"  => kf, 
        "kr"  => kr, 
        "NS0" => NS0,
        "τF"  => τF,
        )
    
    domainfunc = (u, p, t) -> any(x -> x < 0, u)
    u0 = [0., 0., 0., 0., 0., 0., 0., 0.] 
    tspan = (0.0, 1e13) #adequate time to reach steady state? t --> :∞ 
    tvals   = 10 .^range(-1, 13, length = 200)
    f    = ODEFunction(tCSTR4!)
    prob = ODEProblem(f, u0, tspan, par)  
    tCSTRsol = solve(prob, Rodas4(), callback = cb, tstops = [1.0e8], abstol = 1e-11, reltol = 1e-11, isoutofdomain = domainfunc, dtmin = 1e-18)

    #time set for plotting on log x axis
    tCSTRout = tCSTRsol(tvals)
    
    #Output information, function of time.
    FA = tCSTRout[1, :]
    FB = tCSTRout[2, :]
    FC = tCSTRout[3, :]
    FP = tCSTRout[4, :]
    θA = tCSTRout[5, :]
    θB = tCSTRout[6, :]
    θC = tCSTRout[7, :]
    θP = tCSTRout[8, :]
    θV = 1 .- θA .- θB .- θC .- θP
    XA = (FAf .- FA)/FAf
    
    #Specify returns
    return tvals, FA, FB, FC, FP, XA, θA, θB, θC, θP, θV
end

tCSTRmain4 (generic function with 1 method)

In [12]:
sim_pars4 = Dict(
    "NS0"   => 10,                           #moles of active sites in bed
    "DSACT" => [-100, -100, -100, 15, 0],      #J/mol/K
    "DHACT" => [0, 0, 0, 112, 130]*1000,      #J/mol
    "DS"    => [-100, -100, -100, 10, 0],      #J/mol/K
    "DH"    => [-100, -100, -100, 4, -5]*1000, #J/mol
    "CB"    => nothing
)

t4, FA4, FB4, FC4, FP4, XA4, θA4, θB4, θC4, θP4, θV4 = tCSTRmain4(constants, sim_pars4);
tLABEL4 = ["t (s)" "FA (mol/s)" "FB (mol/s)" "FC (mol/s)" "FP (mol/s)" "XA" "θA" "θB" "θC" "θP" "θV"]
tDATA4  = [t4/1e6 FA4*1e6 FB4*1e6 FC4*1e6 FP4 XA4 θA4 θB4 θC4 θP4 θV4]
FLOW_TRANSIENT_4 = vcat(tLABEL4, tDATA4);

io = open("FLOW_TRANSIENT_4.csv", "w")
writedlm(io, FLOW_TRANSIENT_4, ',')
close(io)

@printf("Steady state flowrates of A, B, and C are %3.2E, %3.2E, %3.2E mol/s/mol active site \n", FA4[end], FB4[end], FC4[end])
@printf("Fractional Conversion of A is %3.2E \n", XA4[end])
@printf("Steady State Coverages of A, B, C, P, and V are %3.2E, %3.2E, %3.2E, %3.2E, %3.2E \n", θA4[end], θB4[end], θC4[end], θP4[end], θV4[end])
@printf("The turnover frequency for this reaction is approximately %3.2E mol/s/mol site.", FC4[end])

┌ Warning: Using arrays or dicts to store parameters of different types can hurt performance.
│ Consider using tuples instead.
└ @ SciMLBase C:\Users\jqbon\.julia\packages\SciMLBase\AjIEA\src\performance_warnings.jl:33


Steady state flowrates of A, B, and C are 1.00E+00, 2.00E+00, 7.14E-05 mol/s/mol active site 
Fractional Conversion of A is 7.14E-05 
Steady State Coverages of A, B, C, P, and V are 3.26E-01, 6.63E-01, 1.04E-02, 0.00E+00, 1.67E-10 
The turnover frequency for this reaction is approximately 7.14E-05 mol/s/mol site.

In [13]:
function condition5(u, t, integrator)
    return t == 1.0e8
end

function affect5!(integrator)
    integrator.p["FPf"] = 1.0e-6
end

cb5 = DiscreteCallback(condition5, affect5!)

sim_pars5 = Dict(
    "NS0"   => 10,                           #moles of active sites in bed
    "DSACT" => [-100, -100, -100, 15, 0],    #J/mol/K
    "DHACT" => [0, 0, 0, 112, 130]*1000,     #J/mol
    "DS"    => [-100, -100, -100, 10, 0],    #J/mol/K
    "DH"    => [-100, -100, -100, 4, -5]*1000, #J/mol
    "CB"    => cb5
)

t5, FA5, FB5, FC5, FP5, XA5, θA5, θB5, θC5, θP5, θV5 = tCSTRmain4(constants, sim_pars5);
tLABEL5 = ["t (s)" "FA (mol/s)" "FB (mol/s)" "FC (mol/s)" "FP (mol/s)" "XA" "θA" "θB" "θC" "θP" "θV"]
tDATA5  = [t5/1e6 FA5*1e6 FB5*1e6 FC5*1e6 FP5*1e6 XA5 θA5 θB5 θC5 θP5 θV5]
FLOW_TRANSIENT_5 = vcat(tLABEL5, tDATA5);

io = open("FLOW_TRANSIENT_5.csv", "w")
writedlm(io, FLOW_TRANSIENT_5, ',')
close(io)

@printf("Steady state flowrates of A, B, C, and P are %3.1E, %3.1E, %3.1E, %3.1E mol/s/mol active site \n", FA5[end], FB5[end], FC5[end], FP5[end])
@printf("Fractional Conversion of A is %3.2E \n", XA5[end])
@printf("Steady State Coverages of A, B, C, P, and V are %3.2E, %3.2E, %3.2E, %3.2E, %3.2E \n", θA5[end], θB5[end], θC5[end], θP5[end], θV5[end])
@printf("The turnover frequency for this reaction is approximately %3.2E mol/s/mol site.", FC5[end])

Steady state flowrates of A, B, C, and P are 1.0E+00, 2.0E+00, 6.6E-61, 1.0E-06 mol/s/mol active site 
Fractional Conversion of A is 0.00E+00 
Steady State Coverages of A, B, C, P, and V are 5.23E-53, 1.05E-52, 9.52E-59, 1.00E+00, 0.00E+00 
The turnover frequency for this reaction is approximately 6.56E-61 mol/s/mol site.